# Import libs

In [158]:
!pip install openai

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [42]:
import requests, os, random, time, csv, json
from collections import namedtuple
from bs4 import BeautifulSoup
import pandas as pd
import openai
from graphdatascience import GraphDataScience

In [ ]:
openai.api_key = <key>
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = <pw>

In [43]:
def get_embedding(text_to_embed):
    # Embed a line of text
    response = openai.Embedding.create(
        model= "text-embedding-ada-002",
        input=json.dumps([text_to_embed])
    )
    # Extract the AI output embedding as a list of floats
    embedding = response["data"][0]["embedding"]
    
    return embedding

# Collecting chapters

In [4]:
list_texte = []

list_titre = []

In [5]:
def search_booklets(soup):
    url_base = "https://ithandbook.ffiec.gov"
    #print(url_base)
    tbody = soup.find("tbody")
    #print(tbody)
    rows = tbody.find_all("tr")
    liens = [row.find("a") for row in rows]
    urls = [url_base + lien.get('href') for lien in liens]
    return [url for url in urls if "archived" not in url]

In [6]:
url = "https://ithandbook.ffiec.gov/it-booklets"

response = requests.get(url, verify=False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    all_booklets = search_booklets(soup) 

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
all_booklets

['https://ithandbook.ffiec.gov/it-booklets/audit/',
 'https://ithandbook.ffiec.gov/it-booklets/business-continuity-management/',
 'https://ithandbook.ffiec.gov/it-booklets/development-and-acquisition/',
 'https://ithandbook.ffiec.gov/it-booklets/information-security/',
 'https://ithandbook.ffiec.gov/it-booklets/management/',
 'https://ithandbook.ffiec.gov/it-booklets/architecture-infrastructure-and-operations/',
 'https://ithandbook.ffiec.gov/it-booklets/outsourcing-technology-services/',
 'https://ithandbook.ffiec.gov/it-booklets/retail-payment-systems/',
 'https://ithandbook.ffiec.gov/it-booklets/supervision-of-technology-service-providers/',
 'https://ithandbook.ffiec.gov/it-booklets/wholesale-payment-systems/']

In [8]:
def search_hyperlink(soup):

    url_base = "https://ithandbook.ffiec.gov"
    div = soup.find("div", {"class": "card-body"})
    liens = div.find_all("a")
    lien_href = [lien.get("href") for lien in liens]
    end_article_link = []

    for i in lien_href:
        end_article_link.append(url_base + i)

    return end_article_link

In [9]:
#BLOC DE SCRAPPING

session = requests.Session()

In [10]:
#url = "https://ithandbook.ffiec.gov/it-booklets/wholesale-payment-systems/"
#input("donne moi l'url de l'article de la FFIEC ")#https://ithandbook.ffiec.gov/it-booklets/information-security/introduction/

all_link = {}
 
for booklet_url in all_booklets:
    
    print(booklet_url)
    response = requests.get(booklet_url, verify=False)
    
    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        all_link[booklet_url] = search_hyperlink(soup)

https://ithandbook.ffiec.gov/it-booklets/audit/


/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://ithandbook.ffiec.gov/it-booklets/business-continuity-management/


/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://ithandbook.ffiec.gov/it-booklets/development-and-acquisition/


...

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://ithandbook.ffiec.gov/it-booklets/supervision-of-technology-service-providers/


/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://ithandbook.ffiec.gov/it-booklets/wholesale-payment-systems/


/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
Item = namedtuple('Item', ['link', 'depth', 'position'])
ItemText = namedtuple('Item', ['link', 'depth', 'position','title','text'])

In [12]:
all_link_depth_pos = {}
for k in all_link:
    all_link_depth_pos[k] = [Item(link, link.count('/')-k.count('/'), ix) for ix, link in enumerate(all_link[k])]

In [13]:
all_link_depth_pos_title_text = {}
for k in all_link_depth_pos:
    items = []
    for item in all_link_depth_pos[k]:
        response_article = requests.get(item.link, verify=False)#je vérifie le lien
        if response.status_code == 200:
            soup_doc = BeautifulSoup(response_article.text, "html.parser")
            div_bootle = soup_doc.find("div", {"id" : "bookletcontent"}) #je recupere le texte de l'article 
            # je cherche le texte
            text = div_bootle.get_text()
            text_paragraphs = text.split('\n\n')
            #je cherche le titre
            h3_tags = soup_doc.find("h3")
            title = h3_tags.get_text()
            item_text = ItemText(item.link, item.depth, item.position, title, text_paragraphs)
            items.append(item_text)
    all_link_depth_pos_title_text[k] = items

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

...

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ithandbook.ffiec.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

In [14]:
with open('ffiec_paragraphs_no_embedding.csv', 'w', newline='') as csvfile:
    fieldnames = ['section', 'url', 'depth', 'ordering_ix', 'title', 'paragraph_text', 'paragraph_ordering', 'paragraph_embedding']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for section in all_link_depth_pos_title_text:
        for item in all_link_depth_pos_title_text[section]:
            for ix, paragraph in enumerate(item.text):
                try:
                    writer.writerow({'section':section,
                                    'url':item.link,
                                    'depth':item.depth,
                                    'ordering_ix':item.position,
                                    'title':item.title.strip(),
                                    'paragraph_text':paragraph.strip(),
                                    'paragraph_ordering':ix,
                                    #'paragraph_embedding': item.title.strip() + '\n' + get_embedding(paragraph.strip())
                                    })
                except(e):
                    print(e)
                    print(paragraph.strip())
                    print("**")
                    break


In [180]:
all_link_depth_pos_title_text['https://ithandbook.ffiec.gov/it-booklets/audit/'][11]

Item(link='https://ithandbook.ffiec.gov/it-booklets/audit/risk-assessment-and-risk-based-auditing/', depth=1, position=11, title='\r\n\r\nRisk Assessment and Risk-Based Auditing', text=['', 'Action Summary\nThe board of directors should establish an effective risk-based\naudit function.', "An effective risk-based auditing program will cover all of an\ninstitution's major activities. The frequency and depth of each\narea's audit will vary according to the risk assessment of that\narea. Examiners should determine whether the audit function is\nappropriate for the size and complexity of the institution.\n"])

In [22]:
df = pd.read_csv("ffiec_paragraphs_no_embedding.csv")

In [26]:
df.drop(['paragraph_embedding'], axis=1)

,section,url,depth,ordering_ix,title,paragraph_text,paragraph_ordering
0,https://ithandbook.ffiec.gov/it-booklets/audit/,https://ithandbook.ffiec.gov/it-booklets/audit...,1,0,Introduction,"This ""Audit Booklet"" is one of several booklet...",0
1,https://ithandbook.ffiec.gov/it-booklets/audit/,https://ithandbook.ffiec.gov/it-booklets/audit...,1,0,Introduction,"To determine what risks exist, management shou...",1
2,https://ithandbook.ffiec.gov/it-booklets/audit/,https://ithandbook.ffiec.gov/it-booklets/audit...,1,0,Introduction,Identify areas of greatest IT risk exposure to...,2
3,https://ithandbook.ffiec.gov/it-booklets/audit/,https://ithandbook.ffiec.gov/it-booklets/audit...,1,0,Introduction,The examiner is responsible for evaluating the...,3
4,https://ithandbook.ffiec.gov/it-booklets/audit/,https://ithandbook.ffiec.gov/it-booklets/audit...,1,0,Introduction,Independence of the audit function and its rep...,4
...,...,...,...,...,...,...,...
5900,https://ithandbook.ffiec.gov/it-booklets/whole...,https://ithandbook.ffiec.gov/it-booklets/whole...,1,44,Appendix D: Legal Framework for Interbank Paym...,"State and federal statutes, regulations, and c...",0
5901,https://ithandbook.ffiec.gov/it-booklets/whole...,https://ithandbook.ffiec.gov/it-booklets/whole...,1,45,Appendix E: Federal Reserve Board Payment Syst...,Similar to financial institutions offering ret...,0
5902,https://ithandbook.ffiec.gov/it-booklets/whole...,https://ithandbook.ffiec.gov/it-booklets/whole...,1,46,Appendix F: Payment System Resiliency,"The ""Interagency Paper on Sound Practices to S...",0
5903,https://ithandbook.ffiec.gov/it-booklets/whole...,https://ithandbook.ffiec.gov/it-booklets/whole...,1,46,Appendix F: Payment System Resiliency,Identifying clearing and settlement activities...,1


In [31]:
df['TEST'] = df['depth'] + 1

In [39]:
df = df.drop(['paragraph_embedding'], axis=1)

In [44]:
df['paragraph_embedding'] = df['paragraph_text'].map(get_embedding)

In [46]:
df.to_csv('ffiec_paragraphs_with_embedding.csv')

In [48]:
df['paragraph_embedding'][0]

[-0.00226034177467227,
 0.0014496034709736705,
 -0.006841390393674374,
 -0.03621912747621536,
 -0.02183879166841507,
 -0.003709106706082821,
 -0.018525412306189537,
 -0.02293878048658371,
 -0.004165199585258961,
 -0.03517279401421547,
 0.04839948192238808,
 -0.011724265292286873,
 0.0012643157970160246,
 0.0007998390356078744,
 0.0037124603986740112,
 0.023059509694576263,
 0.026118014007806778,
 -0.00809564534574747,
 0.0026996664237231016,
 -0.01993393339216709,
 -0.038419101387262344,
 -0.010027332231402397,
 -0.03900934010744095,
 -0.0004066267574671656,
 0.0012316179927438498,
 -0.017344936728477478,
 0.01201267633587122,
 -0.0448044016957283,
 -0.011979140341281891,
 0.024977782741189003,
 -0.018297364935278893,
 0.004124956205487251,
 -0.00099434913136065,
 -0.009873065166175365,
 -0.006801147013902664,
 -0.013488270342350006,
 -0.010979760438203812,
 -0.012958398088812828,
 0.016674213111400604,
 0.015279104933142662,
 0.03106796182692051,
 0.004607877694070339,
 -0.02527290023